# Import Libraries and Setting API

In [1]:
import tweepy
import json
import datetime
import warnings
import time
warnings.filterwarnings('ignore')
consumer_key = 'nRO8b2R7tnipKtrFTJVRUru7V'
consumer_secret = 'bK2v3PVVZGktAT2BbviC96Tz9vzjq0ZDj2WLKZ1pmm5aGgu4eW'
access_token = '1098973181468205057-xXfZG8xbtYbXjE6T2s0Ok5J0bKRpsN'
access_token_secret = 'W7muOM4ABrbSH59nHLH43YMeRl3GZCfN1v8FOh1p8tPHh'
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, wait_on_rate_limit = True)

# API 2 function

In [2]:
# SNSUser.accountID, SNSUser.userName, Person.personName OR Team.teamName -> API.user
def anyname2user(anyname):
    user = api.get_user(anyname)
    return user

In [3]:
# API.user -> SNSUser.accountID
def user2accountid(user):
    return user.id

In [4]:
# API.user -> Person.personName OR Team.teamName
def user2name(user):
    return user.name

In [5]:
# API.user -> SNSUser.userName
def user2username(user):
    return user.screen_name

In [6]:
# API.user -> Location.locationName
def user2locationname(user):
    location = ""
    pgcount = 1
    allflag = False
    timenow   = datetime.datetime.now()
    tweetlist = []
    start = time.clock()
    while not allflag:
        tweets = api.user_timeline(user.id, page = pgcount, tweet_mode = 'extended')
        for tweet in tweets:
            if (timenow - tweet.created_at).days < 1:
                tweetlist.append(tweet)
            else:
                 allflag = True
        if not allflag:
            pgcount += 1
        timeused = time.clock() - start
        if timeused > 5:
            break
    for tweet in tweetlist:
        if not tweet.place == None:
            location = tweet.place.__dict__.get("full_name")
            if location != "":
                break
    if location == "":
        location = user.location
    return location

In [7]:
# API.user -> SNSUser.followersCount
def user2followerscount(user):
    return user.followers_count

In [8]:
# API.user -> SNSUser.postsCount
def user2postscount(user):
    return user.statuses_count

In [9]:
# API.post -> Post.postID
def post2postid(tweet):
    return tweet._json.get("id")

In [10]:
# API.post -> Post.SNSUserID
def post2snsuserid(tweet):
    return tweet._json.get("user").get("id")

In [11]:
# API.post -> Post.content
def post2content(tweet):
    return tweet._json.get("full_text")

In [12]:
# API.post -> Post.favoriteCount
def post2favoritecount(tweet):
    return tweet._json.get("favorite_count")

In [13]:
# API.post -> Post.createdTime
def post2createdtime(tweet):
    return tweet.created_at

In [14]:
# API.post -> API.post.location
def post2postlocation(tweet):
    return tweet.place.__dict__.get("full_name")

In [15]:
# API.post -> Tag.tagText(hashtag) list
def post2tagtexthashtaglist(tweet):
    hashtags = tweet._json.get("entities").get("hashtags")
    hashtagList = []
    for hashtag in hashtags:
        hashtagList.append(hashtag.get("text"))
    return hashtagList

In [16]:
# API.post -> Tag.tagText(mention) list
def post2tagtextmentionlist(tweet):
    mentions = tweet._json.get("entities").get("user_mentions")
    mentionList = []
    for mention in mentions:
        mentionList.append(mention.get("screen_name"))
    return mentionList

In [17]:
# SNSUser.accountID, SNSUser.userName, Person.personName OR Team.teamName -> @param number days posts list
def anyname2dayspostlist(anyname, number):
    pgcount = 1
    allflag = False
    timenow   = datetime.datetime.now()
    tweetlist = []
    while not allflag:
        tweets = api.user_timeline(anyname, page = pgcount, tweet_mode = 'extended')
        for tweet in tweets:
            if (timenow - tweet.created_at).days < number:
                tweetlist.append(tweet)
            else:
                 allflag = True
        if not allflag:
            pgcount += 1
        else:
            return tweetlist

In [18]:
# Tag.tagText(hashtag) -> posts list
def tagtexthashtag2postlist(hashtag, number):
    tweets = []
    for tweet in tweepy.Cursor(api.search, q='#' + hashtag, tweet_mode = 'extended').items(number):
        tweets.append(tweet)
    return tweets

In [19]:
# Tag.tagText(mention) -> posts list
def tagtextmention2postlist(mention, number):
    tweets = []
    for tweet in tweepy.Cursor(api.search, q='@' + mention, tweet_mode = 'extended').items(number):
        tweets.append(tweet)
    return tweets

# Make DataFrame

In [20]:
import pandas as pd
Player = pd.DataFrame(columns = ["personID"])
Player_Fan = pd.DataFrame(columns = ["prfID", "PlayerpersonID", "FanpersonID"])
Fan = pd.DataFrame(columns = ["personID"])
Fan_Team = pd.DataFrame(columns = ["frtID", "FanpersonID", "TeamteamID"])
Person = pd.DataFrame(columns = ["personID", "personName", "locationID"])
Team = pd.DataFrame(columns = ["teamID", "teamName", "locationID"])
Person_SNSUser = pd.DataFrame(columns = ["prsID", "PersonpersonID", "SNSUserSNSUserID"])
Location = pd.DataFrame(columns = ["locationID", "locationName"])
Team_SNSUser = pd.DataFrame(columns = ["trsID", "TeamteamID", "SNSUserSNSUserID"])
SNSUser = pd.DataFrame(columns = ["SNSUserID", "SNSType", "accountID", "userName", "followersCount", "postsCount"])
Post = pd.DataFrame(columns = ["SNSpostID", "postID", "SNSUserID", "content", "favoriteCount", "createdTime"])
Post_Tag = pd.DataFrame(columns = ["prtID", "PostSNSpostID", "TagTagID"])
Tag = pd.DataFrame(columns = ["tagID", "tagText", "tagType"])
DomainTag = pd.DataFrame(columns = ["tagID"])
SynonymsTag = pd.DataFrame(columns = ["tagID1", "tagID2"])
MisspellingsTag = pd.DataFrame(columns = ["tagID1", "tagID2"])
TagCatagory_Tag = pd.DataFrame(columns = ["trtID", "TagCatagorycatagoryID", "TagTagID"])
TagCatagory = pd.DataFrame(columns = ["catagoryID", "catagoryName"])

# Data Retriving Function

## User Saving Function

In [21]:
# user saving function
def savesnsuser(user):
    if SNSUser[SNSUser["accountID"].isin([user2accountid(user)])].shape[0] == 0:
        SNSUser.loc[SNSUser.shape[0] + 1] = {"SNSUserID": SNSUser.shape[0] + 1, 
                                             "SNSType": "Twitter", 
                                             "accountID": user2accountid(user), 
                                             "userName": user2username(user), 
                                             "followersCount": user2followerscount(user), 
                                             "postsCount":user2postscount(user)}
    else:
        SNSUser.set_value(SNSUser[SNSUser["accountID"].isin([user2accountid(user)])].loc[:, ["SNSUserID"]].values[0][0], "followersCount", user2followerscount(user))
        SNSUser.set_value(SNSUser[SNSUser["accountID"].isin([user2accountid(user)])].loc[:, ["SNSUserID"]].values[0][0], "postsCount", user2postscount(user))

## Post Saving Function

In [22]:
# post saving function
def savesnspost(post):
    if Post[Post["postID"].isin([post2postid(post)])].shape[0] == 0:
        Post.loc[Post.shape[0] + 1] = {"SNSpostID": Post.shape[0] + 1, 
                                       "postID": post2postid(post), 
                                       "SNSUserID": SNSUser[(SNSUser["accountID"].isin([post2snsuserid(post)])) & (SNSUser["SNSType"] == "Twitter")].loc[:, ["SNSUserID"]].values[0][0], 
                                       "content":post2content(post), 
                                       "favoriteCount":post2favoritecount(post), 
                                       "createdTime":post2createdtime(post)}
        taglist = post2tagtexthashtaglist(post)
        for tag in taglist:
            savesnstag(tag, "hashtag")
        taglist=post2tagtextmentionlist(post)
        for tag in taglist:
            savesnstag(tag, "mention")
    else:
        Post.set_value(Post[Post["postID"].isin([post2postid(post)])].loc[:, ["SNSpostID"]].values[0][0], "favoriteCount", post2favoritecount(post))

## Tag Saving Function

In [23]:
# tag saving function
def savesnstag(tag, tagType):
    if tagType == "hashtag":
        if Tag[(Tag["tagText"].isin([tag])) & (Tag["tagType"] == "hashtag")].shape[0] == 0:
            Tag.loc[Tag.shape[0] + 1] = {"tagID": Tag.shape[0] + 1, 
                                         "tagText": tag, 
                                         "tagType": "hashtag"}
        Post_Tag.loc[Post_Tag.shape[0] + 1] = {"prtID": Post_Tag.shape[0] + 1, 
                                               "PostSNSpostID": Post.shape[0], 
                                               "TagTagID": Tag[(Tag["tagText"].isin([tag])) & (Tag["tagType"] == "hashtag")].loc[:, ["tagID"]].values[0][0]}


    if tagType == "mention":
        if Tag[(Tag["tagText"].isin([tag])) & (Tag["tagType"] == "mention")].shape[0] == 0:
            Tag.loc[Tag.shape[0] + 1] = {"tagID": Tag.shape[0] + 1, 
                                         "tagText": tag, 
                                         "tagType": "mention"}
        Post_Tag.loc[Post_Tag.shape[0] + 1] = {"prtID": Post_Tag.shape[0] + 1, 
                                               "PostSNSpostID": Post.shape[0], 
                                               "TagTagID": Tag[(Tag["tagText"].isin([tag])) & (Tag["tagType"] == "mention")].loc[:, ["tagID"]].values[0][0]}

## Team Saving Function

In [24]:
# team saving function
def saveteam(teamName, userName):
    teamID = 0
    teamIDtemp = 0
    userExistFlag = True
    if SNSUser[SNSUser["accountID"].isin([user2accountid(anyname2user(userName))])].shape[0] == 0:
        userExistFlag = False
    savesnsuser(anyname2user(userName))
    locationName = user2locationname(anyname2user(userName))
    if Location[Location["locationName"].isin([locationName])].shape[0] == 0:
        Location.loc[Location.shape[0] + 1] = {"locationID": Location.shape[0] + 1, 
                                               "locationName": locationName}
    if not userExistFlag:
        Team.loc[Team.shape[0] + 1] = {"teamID": Team.shape[0] + 1, 
                                       "teamName": teamName, 
                                       "locationID": Location[Location["locationName"].isin([locationName])].loc[:, ["locationID"]].values[0][0]}
        teamIDtemp = Team.shape[0]
        teamID = Team.shape[0]
    else:
        teamID = Team[Team["teamName"].isin([teamName])].loc[:, ["teamID"]].values[0][0]
        if (teamIDtemp != 0):
            teamID = teamIDtemp
        Team.set_value(teamID, "locationID", Location[Location["locationName"].isin([locationName])].loc[:, ["locationID"]].values[0][0])
    Team_SNSUser.loc[Team_SNSUser.shape[0] + 1] = {"trsID": Team_SNSUser.shape[0] + 1, 
                                                   "TeamteamID": teamID, 
                                                   "SNSUserSNSUserID": SNSUser[SNSUser["userName"].isin([userName])].loc[:, ["SNSUserID"]].values[0][0]}
    Team_SNSUser.drop_duplicates(subset = ["TeamteamID","SNSUserSNSUserID"], keep='first', inplace=True)

## Player Saving Function

In [25]:
# player saving function
def saveplayer(personName, userName):
    personID = 0
    personIDtemp = 0
    userExistFlag = True
    if SNSUser[SNSUser["accountID"].isin([user2accountid(anyname2user(userName))])].shape[0] == 0:
        userExistFlag = False
    savesnsuser(anyname2user(userName))
    locationName = user2locationname(anyname2user(userName))
    if Location[Location["locationName"].isin([locationName])].shape[0] == 0:
        Location.loc[Location.shape[0] + 1] = {"locationID": Location.shape[0] + 1, 
                                               "locationName": locationName}
    if not userExistFlag:
        Person.loc[Person.shape[0] + 1] = {"personID": Person.shape[0] + 1, 
                                           "personName": personName, 
                                           "locationID": Location[Location["locationName"].isin([locationName])].loc[:, ["locationID"]].values[0][0]}
        Player.loc[Player.shape[0] + 1] = {"personID": Person.shape[0]}
        personIDtemp = Person.shape[0]
        personID = Person.shape[0]
    else:
        personID = Person[Person["personName"].isin([personName])].loc[:, ["personID"]].values[0][0]
        if (personIDtemp != 0):
            personID = personIDtemp
        Person.set_value(Person[Person["personName"].isin([personName])].loc[:, ["personID"]].values[0][0], "locationID", Location[Location["locationName"].isin([locationName])].loc[:, ["locationID"]].values[0][0])
    Person_SNSUser.loc[Person_SNSUser.shape[0] + 1] = {"prsID": Person_SNSUser.shape[0] + 1, 
                                                       "PersonpersonID": personID, 
                                                       "SNSUserSNSUserID": SNSUser[SNSUser["userName"].isin([userName])].loc[:, ["SNSUserID"]].values[0][0]}
    Person_SNSUser.drop_duplicates(subset = ["PersonpersonID","SNSUserSNSUserID"], keep='first', inplace=True)

## save user's posts

In [26]:
# save user's posts
def saveposts(userName, number):
    postlist = anyname2dayspostlist(userName,number)
    for post in postlist:
        savesnspost(post)

## save posts with team #hashtag

In [27]:
# save posts with team #hashtag
def saveteamhashtagposts(hashtag, number):
    postlist = tagtexthashtag2postlist(hashtag, number)
    for post in postlist:
        user = anyname2user(post2snsuserid(post))
        userName = user2username(user)
        personName = user2name(user)
        personID = 0
        userExistFlag = True
        if SNSUser[SNSUser["accountID"].isin([user2accountid(anyname2user(userName))])].shape[0] == 0:
            userExistFlag = False
        personExistFlag = True
        if Person[Person["personName"].isin([personName])].shape[0] == 0: 
            personExistFlag = False
        savesnsuser(anyname2user(userName))
        locationName = user2locationname(anyname2user(userName))
        if Location[Location["locationName"].isin([locationName])].shape[0] == 0:
            Location.loc[Location.shape[0] + 1] = {"locationID": Location.shape[0] + 1, 
                                                   "locationName": locationName}
        if not personExistFlag:
            Person.loc[Person.shape[0] + 1] = {"personID": Person.shape[0] + 1, 
                                               "personName": personName, 
                                               "locationID": Location[Location["locationName"].isin([locationName])].loc[:, ["locationID"]].values[0][0]}
            personID = Person.shape[0]
        else:
            personID = Person[Person["personName"].isin([personName])].loc[:, ["personID"]].values[0][0]
            Person.set_value(Person[Person["personName"].isin([personName])].loc[:, ["personID"]].values[0][0], "locationID", Location[Location["locationName"].isin([locationName])].loc[:, ["locationID"]].values[0][0])
        if (not userExistFlag) | (not personExistFlag):
            Person_SNSUser.loc[Person_SNSUser.shape[0] + 1] = {"prsID": Person_SNSUser.shape[0] + 1, 
                                                               "PersonpersonID": personID, 
                                                               "SNSUserSNSUserID": SNSUser[SNSUser["userName"].isin([userName])].loc[:, ["SNSUserID"]].values[0][0]}
            Person_SNSUser.drop_duplicates(subset=["PersonpersonID","SNSUserSNSUserID"], keep='first', inplace=True)
        if Fan[Fan["personID"].isin([personID])].shape[0] == 0:
            Fan.loc[Fan.shape[0] + 1] = {"personID": personID}
        teamsnsid = SNSUser[SNSUser["userName"].isin([hashtag])].loc[:, ["SNSUserID"]].values[0][0]
        teamteamid = Team_SNSUser[Team_SNSUser["SNSUserSNSUserID"].isin([teamsnsid])].loc[:, ["TeamteamID"]].values[0][0]
        teamName = Team[Team["teamID"].isin([teamteamid])].loc[:, ["teamName"]].values[0][0]
        Fan_Team.loc[Fan_Team.shape[0] + 1] = {"frtID": Fan_Team.shape[0] + 1, 
                                               "FanpersonID": personID, 
                                               "TeamteamID": Team[Team["teamName"].isin([teamName])].loc[:, ["teamID"]].values[0][0]}
        Fan_Team.drop_duplicates(subset=["FanpersonID","TeamteamID"], keep='first', inplace=True)

## save posts with player #hashtag

In [28]:
# save posts with player #hashtag
def saveplayerhashtagposts(hashtag, number):
    postlist = tagtexthashtag2postlist(hashtag, number)
    for post in postlist:
        user = anyname2user(post2snsuserid(post))
        userName = user2username(user)
        personName = user2name(user)
        personID = 0
        userExistFlag = True
        if SNSUser[SNSUser["accountID"].isin([user2accountid(anyname2user(userName))])].shape[0] == 0:
            userExistFlag = False
        personExistFlag = True
        if Person[Person["personName"].isin([personName])].shape[0] == 0: 
            personExistFlag = False
        savesnsuser(anyname2user(userName))
        locationName = user2locationname(anyname2user(userName))
        if Location[Location["locationName"].isin([locationName])].shape[0] == 0:
            Location.loc[Location.shape[0] + 1] = {"locationID": Location.shape[0] + 1, 
                                                   "locationName": locationName}
        if not personExistFlag:
            Person.loc[Person.shape[0] + 1] = {"personID": Person.shape[0] + 1, 
                                               "personName": personName, 
                                               "locationID": Location[Location["locationName"].isin([locationName])].loc[:, ["locationID"]].values[0][0]}
            personID = Person.shape[0]
        else:
            personID = Person[Person["personName"].isin([personName])].loc[:, ["personID"]].values[0][0]
            Person.set_value(Person[Person["personName"].isin([personName])].loc[:, ["personID"]].values[0][0], "locationID", Location[Location["locationName"].isin([locationName])].loc[:, ["locationID"]].values[0][0])
        if (not userExistFlag) | (not personExistFlag):
            Person_SNSUser.loc[Person_SNSUser.shape[0] + 1] = {"prsID": Person_SNSUser.shape[0] + 1, 
                                                               "PersonpersonID": personID, 
                                                               "SNSUserSNSUserID": SNSUser[SNSUser["userName"].isin([userName])].loc[:, ["SNSUserID"]].values[0][0]}
            Person_SNSUser.drop_duplicates(subset=["PersonpersonID","SNSUserSNSUserID"], keep='first', inplace=True)
        if Fan[Fan["personID"].isin([personID])].shape[0] == 0:
            Fan.loc[Fan.shape[0] + 1] = {"personID": personID}
        playersnsid = SNSUser[SNSUser["userName"].isin([hashtag])].loc[:, ["SNSUserID"]].values[0][0]
        personpersonid = Person_SNSUser[Person_SNSUser["SNSUserSNSUserID"].isin([playersnsid])].loc[:, ["PersonpersonID"]].values[0][0]
        personName = Person[Person["personID"].isin([personpersonid])].loc[:, ["personName"]].values[0][0]
        Player_Fan.loc[Player_Fan.shape[0] + 1] = {"prfID": Player_Fan.shape[0] + 1, 
                                                   "PlayerpersonID": Person[Person["personName"].isin([personName])].loc[:, ["personID"]].values[0][0], 
                                                   "FanpersonID": personID}
        Player_Fan.drop_duplicates(subset=["PlayerpersonID","FanpersonID"], keep='first', inplace=True)
        savesnspost(post)

## save posts with @mention

In [29]:
# save posts with @mention
def savementionposts(mention, number):
    postlist = tagtextmention2postlist(mention, number)   
    for post in postlist:
        user = anyname2user(post2snsuserid(post))
        userName = user2username(user)
        personName = user2name(user)
        personID = 0
        userExistFlag = True
        if SNSUser[SNSUser["accountID"].isin([user2accountid(anyname2user(userName))])].shape[0] == 0:
            userExistFlag = False
        personExistFlag = True
        if Person[Person["personName"].isin([personName])].shape[0] == 0: 
            personExistFlag = False
        savesnsuser(anyname2user(userName))
        locationName = user2locationname(anyname2user(userName))
        if Location[Location["locationName"].isin([locationName])].shape[0] == 0:
            Location.loc[Location.shape[0] + 1] = {"locationID": Location.shape[0] + 1, 
                                                   "locationName": locationName}
        if not personExistFlag:
            Person.loc[Person.shape[0] + 1] = {"personID": Person.shape[0] + 1, 
                                               "personName": personName, 
                                               "locationID": Location[Location["locationName"].isin([locationName])].loc[:, ["locationID"]].values[0][0]}
            personID = Person.shape[0]
        else:
            personID = Person[Person["personName"].isin([personName])].loc[:, ["personID"]].values[0][0]
            Person.set_value(Person[Person["personName"].isin([personName])].loc[:, ["personID"]].values[0][0], "locationID", Location[Location["locationName"].isin([locationName])].loc[:, ["locationID"]].values[0][0])
        if (not userExistFlag) | (not personExistFlag):
            Person_SNSUser.loc[Person_SNSUser.shape[0] + 1] = {"prsID": Person_SNSUser.shape[0] + 1, 
                                                               "PersonpersonID": personID, 
                                                               "SNSUserSNSUserID": SNSUser[SNSUser["userName"].isin([userName])].loc[:, ["SNSUserID"]].values[0][0]}
            Person_SNSUser.drop_duplicates(subset = ["PersonpersonID","SNSUserSNSUserID"], keep='first', inplace=True)
        savesnspost(post)

# Saving

## save teams & players

In [30]:
teamSNS = [["Houston Astros", "astros"], 
           ["Detroit Tigers", "tigers"], 
           ["Texas Rangers", "Rangers"]]
playerSNS = [["Alexander David Bregman", "ABREG_1"], 
             ["Carlos Javier Correa", "TeamCJCorrea"], 
             ["José Carlos Altuve", "JoseAltuve27"], 
             ["Nicholas A. Castellanos", "Ncastellano22"], 
             ["Jose Miguel Cabrera", "MiguelCabrera"], 
             ["Jeimer Candelario", "jeimer24C"], 
             ["Elvis Augusto Andrus", "ElvisandrusSS1"], 
             ["Hunter Andrew Pence", "hunterpence"], 
             ["Nomar Shamir Mazara", "NomarMzra26"]]

In [31]:
try:
    counter = 0
    for team in teamSNS:
        saveteam(team[0], team[1])
        counter = counter + 1
        print(counter)
    for player in playerSNS:
        saveplayer(player[0], player[1])
        counter = counter + 1
        print(counter)
    counter = 0
except tweepy.error.TweepError:
    pass

1
2
3
4
5
6
7
8
9
10
11
12


 ## save teams' and players' posts

In [32]:
days = 30
try:
    counter = 0
    for team in teamSNS:
        saveposts(team[1], days)
        counter = counter + 1
        print(counter)
    for player in playerSNS:
        saveposts(player[1], days)
        counter = counter + 1
        print(counter)
    counter = 0
except tweepy.error.TweepError:
    pass

1
2
3
4
5
6
7
8
9
10
11
12


## save tags posts

In [33]:
postscount = 100
try:
    counter = 0
    for team in teamSNS:
        saveteamhashtagposts(team[1], postscount)
        savementionposts(team[1], postscount)
        counter = counter + 1
        print(counter)
    for player in playerSNS:
        saveplayerhashtagposts(player[1], postscount)
        savementionposts(player[1], postscount)
        counter = counter + 1
        print(counter)
    counter = 0
except tweepy.error.TweepError:
    pass

1
2
3
4
5
6
7
8
9
10
11
12


# Show tables

In [34]:
Player.head()

,personID
1,1
2,2
3,3
4,4
5,5


In [35]:
Player_Fan.head()

,prfID,PlayerpersonID,FanpersonID
1,1,3,684
2,2,5,758
3,3,5,759
4,4,5,760
5,5,5,761


In [36]:
Fan.head()

,personID
1,10
2,11
3,12
4,13
5,14


In [37]:
Fan_Team.head()

,frtID,FanpersonID,TeamteamID
1,1,10,1
2,2,11,1
3,3,12,1
4,4,13,1
5,5,14,1


In [38]:
Person.head()

,personID,personName,locationID
1,1,Alexander David Bregman,4
2,2,Carlos Javier Correa,5
3,3,José Carlos Altuve,5
4,4,Nicholas A. Castellanos,6
5,5,Jose Miguel Cabrera,7


In [39]:
Team.head()

,teamID,teamName,locationID
1,1,Houston Astros,1
2,2,Detroit Tigers,2
3,3,Texas Rangers,3


In [40]:
Person_SNSUser.head()

,prsID,PersonpersonID,SNSUserSNSUserID
1,1,1,4
2,2,2,5
3,3,3,6
4,4,4,7
5,5,5,8


In [41]:
Location.head()

,locationID,locationName
1,1,Minute Maid Park
2,2,Comerica Park
3,3,Deep in the Heart of Texas
4,4,"Albuquerque, NM"
5,5,


In [42]:
Team_SNSUser.head()

,trsID,TeamteamID,SNSUserSNSUserID
1,1,1,1
2,2,2,2
3,3,3,3


In [43]:
SNSUser.head()

,SNSUserID,SNSType,accountID,userName,followersCount,postsCount
1,1,Twitter,52803520,astros,1455653,72000
2,2,Twitter,30008146,tigers,1453071,49251
3,3,Twitter,40931019,Rangers,1426982,43627
4,4,Twitter,382708289,ABREG_1,242213,7895
5,5,Twitter,2811330143,TeamCJCorrea,221551,740


In [44]:
Post.head()

,SNSpostID,postID,SNSUserID,content,favoriteCount,createdTime
1,1,1121607683507810304,1,"Final: Indians 2, #Astros 1",111,2019-04-26 02:51:24
2,2,1121598945145147392,1,We know you'd like this in your living room. \...,1068,2019-04-26 02:16:41
3,3,1121582532955230208,1,"Hang it, then bang it. 💥\n\nWe're all square! ...",2829,2019-04-26 01:11:28
4,4,1121579802551435264,1,George doesn't look too frustrated anymore.\n\...,1453,2019-04-26 01:00:37
5,5,1121575278755471360,1,Special message from the @Indians to Uncle Mik...,623,2019-04-26 00:42:38


In [45]:
Post_Tag.head()

,prtID,PostSNSpostID,TagTagID
1,1,1,1
2,2,2,2
3,3,3,2
4,4,4,2
5,5,5,2


In [46]:
Tag.head()

,tagID,tagText,tagType
1,1,Astros,hashtag
2,2,TakeItBack,hashtag
3,3,Indians,mention
4,4,ABREG_1,mention
5,5,ATTSportsNetSW,mention


In [47]:
DomainTag.head()

,tagID


In [48]:
SynonymsTag.head()

,tagID1,tagID2


In [49]:
MisspellingsTag.head()

,tagID1,tagID2


In [50]:
TagCatagory_Tag.head()

,trtID,TagCatagorycatagoryID,TagTagID


In [51]:
TagCatagory.head()

,catagoryID,catagoryName


In [52]:
PlayerList = Player.values.tolist()
Player_FanList = Player_Fan.values.tolist()
FanList = Fan.values.tolist()
Fan_TeamList = Fan_Team.values.tolist()
PersonList = Person.values.tolist()
TeamList = Team.values.tolist()
Person_SNSUserList = Person_SNSUser.values.tolist()
LocationList = Location.values.tolist()
Team_SNSUserList = Team_SNSUser.values.tolist()
SNSUserList = SNSUser.values.tolist()
PostList = Post.values.tolist()
Post_TagList = Post_Tag.values.tolist()
TagList = Tag.values.tolist()
DomainTagList = DomainTag.values.tolist()
SynonymsTagList = SynonymsTag.values.tolist()
MisspellingsTagList = MisspellingsTag.values.tolist()
TagCatagory_TagList = TagCatagory_Tag.values.tolist()
TagCatagoryList = TagCatagory.values.tolist()

# Tagging

## DomainTag

In [53]:
for tag in TagList:
    if tag[2]=="hashtag":
        for player in playerSNS:
            if tag[1] in player[1]:
                DomainTagList.append([tag[0]])
        for team in teamSNS:
            if tag[1] in team[1]:
                DomainTagList.append([tag[0]])
DomainTagList

[[2553], [2571], [2571], [2613], [2656], [2658], [2724], [2738], [2885]]

## SynonymsTag

In [54]:
def tagcount1(tagid):
    counter = 0
    for pt in Post_TagList:
        if pt[2]==tagid:
            counter=counter+1
    return counter
tagcount1(5)

30

In [55]:
def tagcount2(tagid1, tagid2):
    counter=0
    tagid1list = []
    for posttag in Post_TagList:
        if posttag[2]==tagid1:
            tagid1list.append(posttag[1])
    tagid2list = []
    for posttag in Post_TagList:
        if posttag[2]==tagid2:
            tagid2list.append(posttag[1])
    for tagid1 in tagid1list:
        for tagid2 in tagid2list:
            if tagid1==tagid2:
                counter=counter+1
    return counter
tagcount2(1,2)

3

In [79]:
ratioleast = 0.75
countleast = 10
for tag1 in TagList:
    if tag1[2] == "hashtag" and tagcount1(tag1[0])>countleast:    
        for tag2 in TagList:
            if tag2[2] == "hashtag" and tagcount1(tag2[0]>countleast):
                if tag1 != tag2:
                    tag1id = tag1[0]
                    tag2id = tag2[0]
                    f1 = tagcount1(tag1id)
                    f2 = tagcount1(tag2id)
                    f3 = tagcount2(tag1id, tag2id)
                    if f3/f1 > ratioleast or f3/f2 >ratioleast:
                        if tag1id > tag2id:
                            temptagid = tag1id
                            tag1id = tag2id
                            tag2id = temptagid
                        if [tag1id, tag2id] not in SynonymsTagList:
                            SynonymsTagList.append([tag1id, tag2id])

In [80]:
SynonymsTagList

[[1, 55],
 [1, 102],
 [1, 2649],
 [1, 2650],
 [1, 2651],
 [1, 2704],
 [1, 2705],
 [1, 2706],
 [2, 15],
 [2, 133],
 [2, 2195],
 [2, 2476],
 [2, 2477],
 [2, 2608],
 [2, 2658],
 [137, 1791],
 [137, 2215],
 [137, 2216],
 [163, 266],
 [163, 289],
 [163, 500],
 [163, 532],
 [163, 704],
 [163, 708],
 [163, 832],
 [163, 976],
 [163, 1169],
 [163, 1471],
 [163, 2538],
 [163, 2787],
 [163, 2801],
 [255, 1169],
 [255, 2551],
 [255, 2728],
 [255, 2729],
 [255, 2730],
 [255, 2735],
 [255, 2736],
 [255, 2737],
 [255, 2739],
 [255, 2740],
 [255, 2741],
 [255, 2742],
 [255, 2743],
 [255, 2744],
 [255, 2745],
 [255, 2746],
 [255, 2747],
 [255, 2748],
 [882, 2551],
 [882, 2571],
 [882, 2572],
 [882, 2573],
 [882, 2575],
 [882, 2671],
 [882, 2672],
 [882, 2701],
 [882, 2728],
 [882, 2729],
 [882, 2730],
 [882, 2735],
 [882, 2736],
 [882, 2737],
 [882, 2739],
 [882, 2740],
 [882, 2741],
 [882, 2742],
 [882, 2743],
 [882, 2744],
 [882, 2745],
 [882, 2746],
 [882, 2747],
 [882, 2748],
 [882, 2829],
 [882, 2

## MisspellingsTag

### Levenshtein distance calculation function

In [59]:
# Levenshtein distance calculation function
# Reference: https://rosettacode.org/wiki/Levenshtein_distance#Python
def ldistance(s1, s2):
    if len(s1) > len(s2):
        s1, s2 = s2, s1
    distances = range(len(s1) + 1)
    for index2, char2 in enumerate(s2):
        newDistances = [index2 + 1]
        for index1,char1 in enumerate(s1):
            if char1 == char2:
                newDistances.append(distances[index1])
            else:
                newDistances.append(1 + min((distances[index1],
                                             distances[index1 + 1],
                                             newDistances[-1])))
        distances = newDistances
    return distances[-1]

In [60]:
d1ratioleast = 0.30
d2ratioleast = 0.20
d3ratioleast = 0.10
for tag1 in TagList:
    if tag1[2] == "hashtag":    
        for tag2 in TagList:
            if tag2[2] == "hashtag":
                if tag1 != tag2:
                    tag1id=tag1[0]
                    tag2id=tag2[0]
                    f1=tagcount1(tag1id)
                    f2=tagcount1(tag2id)
                    distance=ldistance(tag1[1],tag2[1])
                    if distance<4:
                        if distance==1:
                            ratio=d1ratioleast
                        if distance==2:
                            ratio=d2ratioleast
                        if distance==3:
                            ratio=d3ratioleast
                        if f1/f2<ratio or f2/f1<ratio:
                            if f1>f2:
                                temptag =tag1id
                                tag1id=tag2id
                                tag2id=temptag
                                tempf=f1
                                f1=f2
                                f2=tempf
                            misspt0list=[]
                            for misspt in MisspellingsTagList:
                                misspt0list.append(misspt[0])
                            if not tag1id in misspt0list:
                                 MisspellingsTagList.append([tag1id, tag2id])
                            else:
                                missptori = []
                                for misspt in MisspellingsTagList:
                                    if misspt[0] == tag1id:
                                        missptori = misspt
                                        break
                                tagori1=""
                                tagori2=""
                                for tag in TagList:
                                    if tag[0]==missptori[0]:
                                        tagori1=tag[1]
                                    if tag[0]==missptori[1]:
                                        tagori2=tag[1]
                                f1ori=tagcount1(missptori[0])
                                f2ori=tagcount1(missptori[1])
                                rori=f1ori/f2ori
                                r=f1/f2
                                distanceori=ldistance(tagori1,tagori2)
                                if distance <distanceori:
                                    counter=-1
                                    for misspt in MisspellingsTagList:
                                        counter=counter+1
                                        if misspt[0]==tag1id:
                                            break
                                    MisspellingsTagList[counter]=[tag1id,tag2id]
                                else:
                                    if distance ==distanceori:
                                        if r<rori:
                                            counter=-1
                                            for misspt in MisspellingsTagList:
                                                counter=counter+1
                                                if misspt[0]==tag1id:
                                                    break
                                            MisspellingsTagList[counter]=[tag1id,tag2id]                

In [61]:
MisspellingsTagList

[[37, 1],
 [2613, 1],
 [2689, 1],
 [16, 2],
 [2626, 2],
 [26, 65],
 [140, 137],
 [1817, 137],
 [755, 163],
 [1018, 163],
 [2244, 202],
 [2833, 202],
 [2738, 255],
 [550, 2513],
 [1653, 882],
 [1676, 882],
 [1750, 882],
 [2441, 882],
 [2487, 882],
 [2571, 882],
 [2741, 882],
 [2742, 882],
 [2832, 882],
 [1769, 1480],
 [1707, 1527],
 [1590, 2553],
 [2737, 2513],
 [2531, 2553],
 [2917, 2553],
 [2734, 2692],
 [2724, 2722]]

## TagCatagory

In [62]:
TagCatagoryList=[[1,"hashtag"], 
                 [2,"mention"],
                 [3,"domaintag"],
                 [4,"synonymstag"],
                 [5,"misspellingstag"]]

## TagCatagory_Tag

In [63]:
counter=0
for tag in TagList:
    if tag[2]=="hashtag":
        counter=counter+1
        TagCatagory_TagList.append([counter,1,tag[0]])
    if tag[2]=="mention":
        counter=counter+1
        TagCatagory_TagList.append([counter,2,tag[0]])
    domlist = []
    for dom in DomainTagList:
        domlist.append(dom[0])
    if tag[0] in domlist:
        counter=counter+1
        TagCatagory_TagList.append([counter,3,tag[0]])
    synlist=[]
    for syn in SynonymsTagList:
        synlist.append(syn[0])
    if tag[0] in synlist:
        counter=counter+1
        TagCatagory_TagList.append([counter,4,tag[0]])
    mislist=[]
    for mis in MisspellingsTagList:
        mislist.append(mis[0])
    if tag[0] in mislist:
        counter=counter+1
        TagCatagory_TagList.append([counter,5,tag[0]])

In [64]:
TagCatagory_TagList

[[1, 1, 1],
 [2, 1, 2],
 [3, 2, 3],
 [4, 2, 4],
 [5, 2, 5],
 [6, 2, 6],
 [7, 1, 7],
 [8, 2, 8],
 [9, 2, 9],
 [10, 2, 10],
 [11, 2, 11],
 [12, 1, 12],
 [13, 1, 13],
 [14, 1, 14],
 [15, 1, 15],
 [16, 1, 16],
 [17, 5, 16],
 [18, 2, 17],
 [19, 2, 18],
 [20, 2, 19],
 [21, 2, 20],
 [22, 2, 21],
 [23, 1, 22],
 [24, 2, 23],
 [25, 2, 24],
 [26, 2, 25],
 [27, 1, 26],
 [28, 5, 26],
 [29, 2, 27],
 [30, 2, 28],
 [31, 2, 29],
 [32, 2, 30],
 [33, 2, 31],
 [34, 2, 32],
 [35, 2, 33],
 [36, 2, 34],
 [37, 2, 35],
 [38, 2, 36],
 [39, 1, 37],
 [40, 5, 37],
 [41, 2, 38],
 [42, 2, 39],
 [43, 2, 40],
 [44, 2, 41],
 [45, 1, 42],
 [46, 1, 43],
 [47, 2, 44],
 [48, 2, 45],
 [49, 2, 46],
 [50, 1, 47],
 [51, 2, 48],
 [52, 2, 49],
 [53, 2, 50],
 [54, 2, 51],
 [55, 1, 52],
 [56, 2, 53],
 [57, 2, 54],
 [58, 1, 55],
 [59, 2, 56],
 [60, 2, 57],
 [61, 2, 58],
 [62, 2, 59],
 [63, 1, 60],
 [64, 2, 61],
 [65, 2, 62],
 [66, 1, 63],
 [67, 2, 64],
 [68, 1, 65],
 [69, 2, 66],
 [70, 2, 67],
 [71, 2, 68],
 [72, 1, 69],
 [73, 2, 7

# Tagging dataframe

In [65]:
def list2tuplelist(list):
    tuplelist=[]
    for row in list:
        tuplelist.append(tuple(row))
    return tuplelist

In [66]:
labels=["tagID"]
DomainTag=pd.DataFrame.from_records(list2tuplelist(DomainTagList),columns=labels)

In [67]:
DomainTag.head()

,tagID
0,2553
1,2571
2,2571
3,2613
4,2656


In [82]:
labels=["tagID1","tagID2"]
SynonymsTag=pd.DataFrame.from_records(list2tuplelist(SynonymsTagList),columns=labels)

In [83]:
SynonymsTag.head()

,tagID1,tagID2
0,1,55
1,1,102
2,1,2649
3,1,2650
4,1,2651


In [70]:
labels=["tagID1","tagID2"]
MisspellingsTag=pd.DataFrame.from_records(list2tuplelist(MisspellingsTagList),columns=labels)

In [71]:
MisspellingsTag.head()

,tagID1,tagID2
0,37,1
1,2613,1
2,2689,1
3,16,2
4,2626,2


In [72]:
labels=["trtID","TagCatagorycatagoryID","TagTagID"]
TagCatagory_Tag=pd.DataFrame.from_records(list2tuplelist(TagCatagory_TagList),columns=labels)

In [73]:
TagCatagory_Tag.head()

,trtID,TagCatagorycatagoryID,TagTagID
0,1,1,1
1,2,1,2
2,3,2,3
3,4,2,4
4,5,2,5


In [74]:
labels=["catagoryID","catagoryName"]
TagCatagory=pd.DataFrame.from_records(list2tuplelist(TagCatagoryList),columns=labels)

In [75]:
TagCatagory.head()

,catagoryID,catagoryName
0,1,hashtag
1,2,mention
2,3,domaintag
3,4,synonymstag
4,5,misspellingstag


# to csv

In [84]:
#Player = pd.DataFrame(columns = ["personID"])
Player.to_csv(r'Player.csv',index=False, encoding="utf-8-sig")
#Player_Fan = pd.DataFrame(columns = ["prfID", "PlayerpersonID", "FanpersonID"])
Player_Fan.to_csv(r'Player_Fan.csv',index=False, encoding="utf-8-sig")
#Fan = pd.DataFrame(columns = ["personID"])
Fan.to_csv(r'Fan.csv',index=False, encoding="utf-8-sig")
#Fan_Team = pd.DataFrame(columns = ["frtID", "FanpersonID", "TeamteamID"])
Fan_Team.to_csv(r'Fan_Team.csv',index=False, encoding="utf-8-sig")
#Person = pd.DataFrame(columns = ["personID", "personName", "locationID"])
Person.to_csv(r'Person.csv',index=False, encoding="utf-8-sig")
#Team = pd.DataFrame(columns = ["teamID", "teamName", "locationID"])
Team.to_csv(r'Team.csv',index=False, encoding="utf-8-sig")
#Person_SNSUser = pd.DataFrame(columns = ["prsID", "PersonpersonID", "SNSUserSNSUserID"])
Person_SNSUser.to_csv(r'Person_SNSUser.csv',index=False, encoding="utf-8-sig")
#Location = pd.DataFrame(columns = ["locationID", "locationName"])
Location.to_csv(r'Location.csv',index=False, encoding="utf-8-sig")
#Team_SNSUser = pd.DataFrame(columns = ["trsID", "TeamteamID", "SNSUserSNSUserID"])
Team_SNSUser.to_csv(r'Team_SNSUser.csv',index=False, encoding="utf-8-sig")
#SNSUser = pd.DataFrame(columns = ["SNSUserID", "SNSType", "accountID", "userName", "followersCount", "postsCount"])
SNSUser.to_csv(r'SNSUser.csv',index=False, encoding="utf-8-sig")
#Post = pd.DataFrame(columns = ["SNSpostID", "postID", "SNSUserID", "content", "favoriteCount", "createdTime"])
Post.to_csv(r'Post.csv',index=False, encoding="utf-8-sig")
#Post_Tag = pd.DataFrame(columns = ["prtID", "PostSNSpostID", "TagTagID"])
Post_Tag.to_csv(r'Post_Tag.csv',index=False, encoding="utf-8-sig")
#Tag = pd.DataFrame(columns = ["tagID", "tagText", "tagType"])
Tag.to_csv(r'Tag.csv',index=False, encoding="utf-8-sig")
#DomainTag = pd.DataFrame(columns = ["tagID"])
DomainTag.to_csv(r'DomainTag.csv',index=False, encoding="utf-8-sig")
#SynonymsTag = pd.DataFrame(columns = ["tagID1", "tagID2"])
SynonymsTag.to_csv(r'SynonymsTag.csv',index=False, encoding="utf-8-sig")
#MisspellingsTag = pd.DataFrame(columns = ["tagID1", "tagID2"])
MisspellingsTag.to_csv(r'MisspellingsTag.csv',index=False, encoding="utf-8-sig")
#TagCatagory_Tag = pd.DataFrame(columns = ["trtID", "TagCatagorycatagoryID", "TagTagID"])
TagCatagory_Tag.to_csv(r'TagCatagory_Tag.csv',index=False, encoding="utf-8-sig")
#TagCatagory = pd.DataFrame(columns = ["catagoryID", "catagoryName"])
TagCatagory.to_csv(r'TagCatagory.csv',index=False, encoding="utf-8-sig")